<a href="https://colab.research.google.com/github/williamcarp/Equity-Derivative-Models/blob/main/Code/JR_Binomial_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jarrow-Rudd (JR) Binomial Tree Model

Author: Will Carpenter

Date Created: April 1st, 2021

Github Path: Code/JR_Binomial_Tree.ipynb

Implementation of the Jarrow-Rudd binomial tree model for pricing equity derivatives. 

Runtime complexity: O(n^2)

Space complexity: O(n)

In [ ]:
import math 
import numpy as np 
import matplotlib.pyplot as plt
from numpy import genfromtxt
from scipy.stats import norm # Cumulative normal distribution
from scipy.optimize import fsolve # non-linear root-finding

In [ ]:
def jr_binomial_tree(S, K, r, T, t, v, style, type):

    # S  : spot price 
    # K  : strike 
    # r  : riskless rate
    # T  : maturity (in yrs.)
    # t  : number of steps 
    # v  : annualized volatility

    # style : 'amer' or 'euro'
    # type  : 'call' or 'put

    # Calculate time increment 
    dt = T / t

    # Set type of option 
    if type=='call': x = 1 
    if type=='put' : x = -1 

    # Initialize tree
    jrTree        = np.empty((t+1,1))
    jrTree[:]     = np.nan

    # Initialize tree parameters 
    u = math.exp((r - v**2/2)*dt + v*math.sqrt(dt))
    d = math.exp((r - v**2/2)*dt - v*math.sqrt(dt))
    p = 1/2

    for row in range(0,t+1):
        St = S*u**(t-row)*d**(row)
        jrTree[row, 0] = max(x*St - x*K, 0)

    for col in range(t-1, -1, -1):
        for row in range(0, col+1):

            # Determine value of early excercise 
            if style=='euro':   earlyExercise = np.NINF
            if style=='amer':   earlyExercise = x*S*u**(col-row)*d**(row) - x*K

            # Backward iteration from previous nodes
            Pu = jrTree[row, 0]
            Pd = jrTree[row+1, 0]
            
            # Determine continuation value
            continuation = math.exp(-r*dt)*(p*Pu + (1-p)*Pd)

            # Calcuate price on at tree node
            jrTree[row, 0] = max(continuation, earlyExercise)

    return jrTree[0,0]
